# Basic Examples of einmesh

This notebook demonstrates the core functionality of einmesh through simple examples. We'll explore:

1. Creating basic 2D meshgrids using linear spaces
2. Working with 3D grids using different space types (LogSpace and UniformDistribution)
3. Using repeated dimensions in patterns

einmesh provides an intuitive, einops-style syntax for creating multi-dimensional meshgrids, making it easy to define complex coordinate systems with just a few lines of code.


In [ ]:
import matplotlib.pyplot as plt

from einmesh import (
    LinSpace,  # Spaces to create the meshgrid
    LogSpace,
    UniformDistribution,
)

# To use a specific backend, just import the einmesh function
from einmesh.numpy import einmesh
# from einmesh.jax import einmesh
# from einmesh.torch import einmesh

The first thing to know is the use of einstein notation to create the shapes of the meshgrid that is being generated by einmesh. An einmesh pattern consists of space separated names ("x y z"), parenthesis("(x y) z"), stars ("* x y z") and ellipsis ("* x ..."). 

- Names express which dimension should correspond to a keyword Space argument 
- Parenthesis flattens axis into each other, just like einops.rearrange
- A star tells einmesh to stack the outputs into the dimension of the star, instead of returning a tuple
- Ellipsis allows for sampling of spaces that are not named in the keyword arguments, but instead are passed as regular arguments

In the below example we showcase this by creating a 2D square mesh between 0 and 1, with 10 points in each dimension.

In [ ]:
x_coords, y_coords = einmesh("x y", x=LinSpace(start=0, end=1, num=10), y=LinSpace(start=0, end=1.1, num=10))
print(x_coords.shape, y_coords.shape)

plt.figure(figsize=(5, 4))
plt.scatter(x_coords, y_coords)
plt.xlabel("x")
plt.ylabel("y")
plt.title("2D Mesh Grid")
plt.show()

By changing the space type, we change the type of sampling

In [ ]:
lin_coords, log_coords = einmesh(
    "lin log", lin=LinSpace(start=0, end=1, num=10), log=LogSpace(start=0, end=1, num=10, base=10)
)
print(lin_coords.shape, log_coords.shape)
plt.figure(figsize=(5, 4))
plt.scatter(lin_coords, log_coords)
plt.xlabel("lin")
plt.ylabel("log")
plt.title("2D Mesh Grid with log space in y")
plt.show()

If one wants to have two dimension that are identically sampled, one can simply repeat an axis in the pattern. 

In [ ]:
x_coords_repeated, y_coords_repeated = einmesh("x x", x=LinSpace(start=0, end=1, num=10))
print(x_coords_repeated.shape, y_coords_repeated.shape)

If you want the meshgrid to return a stacked tensor, you can use the * operator. The meshgrid will be stacked in the dimensions in which the * is located in the pattern.

In [ ]:
coords_stacked_start = einmesh("* x x", x=LinSpace(start=0, end=1, num=10))
print(f"{coords_stacked_start.shape=}")
coords_stacked_mid = einmesh("x * x", x=LinSpace(start=0, end=1, num=10))
print(f"{coords_stacked_mid.shape=}")
coords_stacked_end = einmesh("x x *", x=LinSpace(start=0, end=1, num=10))
print(f"{coords_stacked_end.shape=}")

In style with einops.rearrange, axis can be combined (flattened), by using a parenthesis.

In [ ]:
coords_flattened = einmesh("* (x y)", x=LinSpace(0, 1, num=10), y=LinSpace(0, 1, num=10))
print(f"{coords_flattened.shape=}")
everything_flattened = einmesh("(* x y)", x=LinSpace(0, 1, num=10), y=LinSpace(0, 1, num=10))
print(f"{everything_flattened.shape=}")

The grids that are generated by einmesh be of arbitrary dimensionality, and arbitrary combination of spaces. So if you want to generate a 9D grid, it is as easy as changing the pattern.

In [ ]:
nine_d_grid = einmesh(
    "* x y z w v u t s r",
    x=LinSpace(start=0, end=1, num=1),
    y=LinSpace(start=0, end=1, num=2),
    z=LogSpace(start=0, end=1, num=3),
    w=LinSpace(start=0, end=1, num=4),
    v=LogSpace(start=0, end=1, num=5),
    u=LinSpace(start=0, end=1, num=6),
    t=LinSpace(start=0, end=1, num=7),
    s=UniformDistribution(low=0, high=1, num=8),
    r=LinSpace(start=0, end=1, num=9),
)
print(f"{nine_d_grid.shape=}")

And you can arbitrarily combine axis as you want

In [ ]:
nine_d_grid = einmesh(
    "x (y z w) v (u t) (* s) r",
    x=LinSpace(start=0, end=1, num=1),
    y=LinSpace(start=0, end=1, num=2),
    z=LogSpace(start=0, end=1, num=3),
    w=LinSpace(start=0, end=1, num=4),
    v=LogSpace(start=0, end=1, num=5),
    u=LinSpace(start=0, end=1, num=6),
    t=LinSpace(start=0, end=1, num=7),
    s=UniformDistribution(low=0, high=1, num=8),
    r=LinSpace(start=0, end=1, num=9),
)
print("Shape should be (x, y*z*w, v, u*t, stacked_dim*s, r)")
print(f"i.e Shape should be {1, 2 * 3 * 4, 5, 6 * 7, 9 * 8, 9}")
print(f"{nine_d_grid.shape=}")

In [ ]:
spaces = [LinSpace(1, 1, 5)] * 8

nine_d_grid = einmesh("* ...", *spaces)
print(nine_d_grid.shape)